In [1]:
# reddit_scraper.py
import praw
from datetime import datetime
from utils import save_to_csv

def scrape_reddit():
    print("\n=== Starting Reddit Scraper ===")
    try:
        reddit = praw.Reddit(
            client_id="WvaVwQIaAz9bIr8q7j0Arw",
            client_secret="H94fxb2g4vTaAK8ZECDwwquOnWnL8w",
            user_agent="FOMO_Scraper/1.0"
        )
        print("Successfully connected to Reddit API")
    except Exception as e:
        print(f"Error connecting to Reddit API: {str(e)}")
        return

    data_list = []
    subreddits = ['FOMO', 'socialanxiety', 'Anxiety', 'mentalhealth']
    
    for subreddit in subreddits:
        try:
            print(f"\nScraping r/{subreddit}...")
            posts = reddit.subreddit(subreddit).search('FOMO', limit=500)
            post_count = 0
            comment_count = 0
            
            for post in posts:
                # Add post data
                data_list.append([
                    datetime.fromtimestamp(post.created_utc),
                    'reddit_post',
                    post.title + ' ' + post.selftext,
                    post.author.name if post.author else 'deleted',
                    post.score
                ])
                post_count += 1

                # Get comments
                post.comments.replace_more(limit=None)  # Expand all comment trees
                for comment in post.comments.list():
                    try:
                        data_list.append([
                            datetime.fromtimestamp(comment.created_utc),
                            'reddit_comment',
                            comment.body,
                            comment.author.name if comment.author else 'deleted',
                            comment.score
                        ])
                        comment_count += 1
                    except Exception as e:
                        print(f"Error processing comment: {str(e)}")
                        continue

            print(f"Collected {post_count} posts and {comment_count} comments from r/{subreddit}")
        except Exception as e:
            print(f"Error scraping r/{subreddit}: {str(e)}")
    
    save_to_csv(data_list, 'reddit')

if __name__ == "__main__":
    scrape_reddit()


=== Starting Reddit Scraper ===
Successfully connected to Reddit API

Scraping r/FOMO...
Collected 51 posts and 88 comments from r/FOMO

Scraping r/socialanxiety...
Collected 108 posts and 972 comments from r/socialanxiety

Scraping r/Anxiety...
Collected 188 posts and 769 comments from r/Anxiety

Scraping r/mentalhealth...
Collected 147 posts and 276 comments from r/mentalhealth
